In [1]:
import pandas as pd
import datetime

In [2]:
actors = pd.read_csv('../Tables/Actors.csv')
directors = pd.read_csv('../Tables/Directors.csv')
genres = pd.read_csv('../Tables/Genres.csv')
movies = pd.read_csv('../Tables/Movie_Data.csv')
studios = pd.read_csv('../Tables/Studios.csv')
titles = pd.read_csv('../Tables/titleNid.csv')

In [3]:
actors.count()

name                3818
actor_id            3818
gender              3818
character           3802
actor_popularity    3818
movie_id            3818
dtype: int64

In [4]:
actor_mean = actors.groupby('movie_id', sort = False).mean()
actor_mean = actor_mean[['actor_popularity']]
actor_mean

,actor_popularity
movie_id,
566525,6.738895
580489,4.363038
370172,4.164382
524434,5.096703
744275,4.204667
...,...
303857,3.765667
565028,1.431400
400160,6.898650


In [5]:
director_mean = directors.groupby('movie_id', sort = False).mean()
director_mean = director_mean[['director_popularity']]
director_mean 

,director_popularity
movie_id,
566525,5.119
580489,13.077
370172,4.167
524434,2.397
744275,2.580
...,...
303857,0.600
565028,0.600
400160,2.717


In [6]:
count = studios.studio_name.value_counts()
replace = list(count[count <= 3].index)
replace
for studio in replace:
    studios.studio_name = studios.studio_name.replace(studio, "Other")
studios.studio_name.value_counts()

Other                    212
Warner Bros. Pictures     13
Columbia Pictures         11
Sony Pictures             11
Marvel Studios             9
Universal Pictures         8
Heyday Films               8
Marvel Entertainment       6
Walt Disney Pictures       6
Pascal Pictures            5
Metro-Goldwyn-Mayer        5
New Line Cinema            4
Name: studio_name, dtype: int64

In [7]:
studios


,studio_id,studio_name,origin_country,movie_id
0,420.0,Marvel Studios,US,566525
1,7505.0,Marvel Entertainment,US,580489
2,84041.0,Pascal Pictures,US,580489
3,5.0,Columbia Pictures,US,580489
4,34.0,Sony Pictures,US,580489
...,...,...,...,...
293,12.0,New Line Cinema,US,72545
294,1836.0,Other,NaN,72545
295,1632.0,Other,US,15092
296,126.0,Other,US,15092


In [8]:
grouped_df = studios.groupby('movie_id',sort = False)
grouped_lists = grouped_df['studio_name'].agg(lambda column: ", ".join(column))
grouped_lists = grouped_lists.reset_index(name='studio_name')
grouped_lists

,movie_id,studio_name
0,566525,Marvel Studios
1,580489,"Marvel Entertainment, Pascal Pictures, Columbi..."
2,370172,"Other, Metro-Goldwyn-Mayer, Other, Universal P..."
3,524434,Marvel Studios
4,744275,"Other, Other, Other, Other, Other"
...,...,...
69,303857,"Other, Other, Other"
70,565028,"Metro-Goldwyn-Mayer, Other, Other, Universal P..."
71,400160,"Other, Other, Other, Other, Other, Other"
72,72545,"Other, New Line Cinema, Other"


In [9]:
holder = []
for names in grouped_lists['studio_name']:
    holder.append(names.split(","))
# movies['studio_list'] = holder
# movies
grouped_lists['studio_list'] = holder
grouped_lists = grouped_lists.drop('studio_name', 1)

/var/folders/61/jyjwflp16vv8tst5lfm0g4xr0000gn/T/ipykernel_9375/3084318855.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  grouped_lists = grouped_lists.drop('studio_name', 1)


In [10]:
movies['studios'] = grouped_lists['studio_list']
movies['release_date'] = pd.to_datetime(movies['release_date'], format= '%Y-%m-%dT')
movies.dtypes
                                        

movie_id                     int64
title                       object
genre_ids                   object
release_date        datetime64[ns]
movie_popularity           float64
vote_average               float64
vote_count                   int64
budget                       int64
revenue                      int64
rating                      object
studios                     object
dtype: object

In [11]:
genres.set_index(genres['id'],inplace = True)
genres = genres.drop('id',1)
genres

/var/folders/61/jyjwflp16vv8tst5lfm0g4xr0000gn/T/ipykernel_9375/65170182.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  genres = genres.drop('id',1)


,name
id,
28,Action
12,Adventure
16,Animation
35,Comedy
80,Crime
99,Documentary
18,Drama
10751,Family
14,Fantasy


In [12]:
genres.loc[28]['name']

'Action'

In [13]:
genre_title = []

for ls in movies.genre_ids:
    holder = []
    for id in eval(ls):
        holder.append(genres.loc[id]['name'])
    genre_title.append(holder)
genre_title

[['Action', 'Adventure', 'Fantasy'],
 ['Science Fiction', 'Action', 'Adventure'],
 ['Adventure', 'Action', 'Thriller'],
 ['Action', 'Adventure', 'Science Fiction'],
 ['Romance', 'Drama'],
 ['Science Fiction', 'Adventure'],
 ['Comedy', 'Action', 'Adventure', 'Science Fiction'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Science Fiction', 'Action'],
 ['Action', 'Adventure'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Animation', 'Action', 'Adventure', 'Fantasy'],
 ['Animation', 'Comedy', 'Adventure', 'Family'],
 ['Action', 'Thriller', 'Crime'],
 ['Horror', 'Mystery', 'Thriller'],
 ['Action', 'Adventure', 'Science Fiction'],
 ['Animation', 'Comedy', 'Family', 'Fantasy'],
 ['Adventure', 'Fantasy', 'Comedy', 'Action'],
 ['Action', 'Crime', 'Thriller'],
 ['Horror', 'Thriller'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Action', 'Adventure', 'Science Fiction'],
 ['Action', 'Adventure', 'Science Fiction', 'Drama'],
 ['Fantasy', 'Action', 'Adventure'],
 ['Animation', 'Family', 'Comedy', 'Science Fictio

In [14]:
movies['genres'] = genre_title
movies = movies.drop('genre_ids',1)

/var/folders/61/jyjwflp16vv8tst5lfm0g4xr0000gn/T/ipykernel_9375/2575431064.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies = movies.drop('genre_ids',1)


In [15]:
movies.head()

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating,studios,genres
0,566525,Shang-Chi and the Legend of the Ten Rings,2021-09-01,5884.885,7.9,3428,150000000,430238384,PG-13,[Marvel Studios],"[Action, Adventure, Fantasy]"
1,580489,Venom: Let There Be Carnage,2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13,"[Marvel Entertainment, Pascal Pictures, Colu...","[Science Fiction, Action, Adventure]"
2,370172,No Time to Die,2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13,"[Other, Metro-Goldwyn-Mayer, Other, Univers...","[Adventure, Action, Thriller]"
3,524434,Eternals,2021-11-03,1746.171,7.1,1170,200000000,336000000,PG-13,[Marvel Studios],"[Action, Adventure, Science Fiction]"
4,744275,After We Fell,2021-09-01,1710.038,7.2,956,14000000,19000000,R,"[Other, Other, Other, Other, Other]","[Romance, Drama]"


In [16]:
ml_table = movies[['movie_id','release_date','movie_popularity', 'vote_average', 'vote_count','budget','revenue', 'rating', 'genres','studios']]
ml_table = ml_table.merge(actor_mean,on='movie_id', how = 'inner')
ml_table = ml_table.merge(director_mean,on='movie_id', how = 'inner')
ml_table = ml_table.drop('movie_id',1)
ml_table

/var/folders/61/jyjwflp16vv8tst5lfm0g4xr0000gn/T/ipykernel_9375/1964443998.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ml_table = ml_table.drop('movie_id',1)


,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating,genres,studios,actor_popularity,director_popularity
0,2021-09-01,5884.885,7.9,3428,150000000,430238384,PG-13,"[Action, Adventure, Fantasy]",[Marvel Studios],6.738895,5.119
1,2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13,"[Science Fiction, Action, Adventure]","[Marvel Entertainment, Pascal Pictures, Colu...",4.363038,13.077
2,2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13,"[Adventure, Action, Thriller]","[Other, Metro-Goldwyn-Mayer, Other, Univers...",4.164382,4.167
3,2021-11-03,1746.171,7.1,1170,200000000,336000000,PG-13,"[Action, Adventure, Science Fiction]",[Marvel Studios],5.096703,2.397
4,2021-09-01,1710.038,7.2,956,14000000,19000000,R,"[Romance, Drama]","[Other, Other, Other, Other, Other]",4.204667,2.580
...,...,...,...,...,...,...,...,...,...,...,...
69,2015-04-18,192.039,6.8,1246,5000000,61768190,NR,"[Action, Animation, Science Fiction]","[Other, Other, Other]",3.765667,0.600
70,2021-08-25,188.391,6.3,615,25000000,77389310,R,"[Horror, Thriller]","[Metro-Goldwyn-Mayer, Other, Other, Univers...",1.431400,0.600
71,2020-08-14,186.863,7.7,2351,60000000,4700000,PG,"[Family, Animation, Fantasy, Adventure, Comedy]","[Other, Other, Other, Other, Other, Other]",6.898650,2.717
72,2012-01-19,185.102,6.1,3312,79000000,355692760,PG,"[Adventure, Action, Science Fiction]","[Other, New Line Cinema, Other]",11.495923,1.425


In [17]:
ml_table.to_csv('Test_ML.csv', index = False)